In [2]:
import pandas as pd
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [3]:
import pandas as pd

data = pd.read_csv('dataset.csv')
data.head()

,word,hyperonym,text,url
0,ГЛАДИОЛУС,растение,"Цветущее растение гладиолус (Gladiolus), еще и...",https://rastenievod.com/gladiolusy.html
1,ГЛАДИОЛУС,цветы,"Гладиолусы — цветы, которые любят солнце, поэт...",https://www.nur.kz/1789297-cvetok-gladiolus-po...
2,ГЛАЗУНЬЯ,метод приготовления яйца,Яичница глазунья — мой самый любимый метод при...,http://hellokitchen.ru/7108/kak-gotovit-yaichn...
3,ГЛАЗУНЬЯ,завтрак,Яичница глазунья - неотъемлемая часть вкусного...,https://vpuzo.com/recepty-vtoryh-blyud/yaichni...
4,ГЛАЗУНЬЯ,блюдо,Яичница глазунья - очень питательное блюдо,http://findfood.ru/recept/yaichnica-glazunya


In [13]:
START = rule(
    gram('NOUN')
)
START_1 = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)
START_2 = rule(
    (
        rule(gram('ADJF')),
        rule(gram('NOUN')),
        rule(gram('NOUN'))
    ))
START_3 = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))))


In [14]:
MID = rule(eq('—'))

In [15]:
END = rule(
         gram('ADJF').repeatable(),
        gram('NOUN')
    )
END_1 = rule(
         gram('ADJF'),
     rule(gram('NOUN')))

END_2 = rule(
    gram('NOUN'))

In [16]:
def get_hyper(row):
    main_word = row['word']
    text = row['text']
    
    HYPONYM = eq(utils.deaccent(main_word.lower()))
    RULE = or_(
           rule(START, MID, END),
           rule(START_1, MID, END), 
           rule(START_1,HYPONYM, MID, END), 
           rule(START, MID, END_2),
           rule(START_3, MID)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    text = re.sub(r'\(.+?\)', '', text)
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps

In [18]:
data['dec_hyper'] = data.apply(lambda row: get_hyper(row), axis=1)

ГЛАДИОЛУС []
ГЛАДИОЛУС [['гладиолусы', '—', 'цветы']]
ГЛАЗУНЬЯ [['яичница', 'глазунья', '—', 'мои', 'самыи']]
ГЛАЗУНЬЯ []
ГЛАЗУНЬЯ []
ГЛАЗУНЬЯ []
ГЛАЗУНЬЯ []
ГЛИКОГЕН []
ГЛИКОГЕН []
ГЛИКОГЕН []
ГЛИКОГЕН []
ГЛИССЕР [['глиссер', '—', 'легкое', 'быстроходное', 'судно']]
ГЛИССЕР []
ГЛОССАРИЙ [['глоссарии', '—', 'словарь']]
ГЛОССАРИЙ []
ГЛУТАМАТ [['глутамат', 'натрия', '—', 'мононатриевая', 'соль']]
ГЛУТАМАТ [['глутамат', 'натрия', '—', 'мононатриевая', 'соль']]
ГЛУТАМАТ []
ГЛУТАМАТ []
ГЛЯЦИОЛОГ [['гляциологи', '—']]
ГЛЯЦИОЛОГ []
ГЛЯЦИОЛОГ []
ГЛЯЦИОЛОГ []
ГОБОИСТ [['подъемов', '—', 'россиискии']]
ГОБОИСТ [['куклес', '—', 'россиискии']]
ГОЛАВЛЬ [['головень', '—', 'пресноводная', 'рыба']]
ГОЛАВЛЬ []
ГОЛАВЛЬ []
ГОЛАВЛЬ []
ГОЛАВЛЬ []
